In [1]:
import numpy as np                      
import pandas as pd                     
import os                               
import cv2      
import tensorflow
import matplotlib.pyplot as plt         
from tqdm.notebook import tqdm          
from sklearn.utils import shuffle    

import requests
import json


In [2]:
foldernames = os.listdir('./training-data')
categories = []
files = []
i = 0
for k, folder in enumerate(foldernames):
    filenames = os.listdir("./training-data/" + folder);
    for file in filenames:
        files.append("./training-data/" + folder + "/" + file)
        categories.append(k)
        
df = pd.DataFrame({
    'filename': files,
    'category': categories
})
train_df = pd.DataFrame(columns=['filename', 'category'])
for i in range(10):
    train_df = train_df.append(df[df.category == i].iloc[:500,:])

train_df.head()
train_df = train_df.reset_index(drop=True)
train_df

,filename,category
0,./training-data/spider/OIP-4Xc2xJTCLn4FSQmQ08M...,0
1,./training-data/spider/OIP-726qaS3uHC3IGduUTTV...,0
2,./training-data/spider/OIP-04fhoeLUsHSQ7MackFT...,0
3,./training-data/spider/OIP-FXgDq6uJO8Bt5wP11db...,0
4,./training-data/spider/OIP-2YRyN4GnEn_8Efun_3z...,0
...,...,...
4995,./training-data/horse/OIP-oap9gpcs2975WMby8jvL...,9
4996,./training-data/horse/OIP-LaMTHLyJxpIBseRgD_ne...,9
4997,./training-data/horse/OIP-jdO7QaRkH8dUnVXGC3Dt...,9
4998,./training-data/horse/OIP-3bZ04VOtOB0Wr_XGJsG7...,9


In [3]:
y = train_df['category']
x = train_df['filename']
y = train_df['category']

x, y = shuffle(x, y, random_state=8) 

In [4]:
def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

images = []
with tqdm(total=len(train_df)) as pbar:
    for i, file_path in enumerate(train_df.filename.values):
        img = cv2.imread(file_path)
        print(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        img = centering_image(cv2.resize(img, dsize=tile_size))

        img = img[16:240, 16:240]
        images.append(img)
        pbar.update(1)

images = np.array(images)

./training-data/spider/OIP-4Xc2xJTCLn4FSQmQ08M7NAHaFa.jpeg
./training-data/spider/OIP-726qaS3uHC3IGduUTTVy_QHaFj.jpeg
./training-data/spider/OIP-04fhoeLUsHSQ7MackFTF_AHaHa.jpeg
./training-data/spider/OIP-FXgDq6uJO8Bt5wP11db47gHaFD.jpeg
./training-data/spider/OIP-2YRyN4GnEn_8Efun_3zfygHaHa.jpeg
./training-data/spider/OIP-NKJXhMo0G_pLhekfFVzX3QHaEn.jpeg
./training-data/spider/OIP-W_7-EaeajgofC-7oOwhSnwEMEs.jpeg
./training-data/spider/OIP-aI0GVerH2CONbgLEn_c1RQHaFS.jpeg
./training-data/spider/OIP-XqA4D3bfoeGFjG54UBsCMwHaEK.jpeg
./training-data/spider/OIP-fDtauUI9EoRxoR3xuAfw0AHaGP.jpeg
./training-data/spider/OIP-u7-KgvBFflFZvXSbmtGtsAEMDo.jpeg
./training-data/spider/OIP-DX4dztiGvYIbEeknoXlt_gHaHa.jpeg
./training-data/spider/OIP-bT5FOljgLy6vbTSZ5j38_QHaFj.jpeg
./training-data/spider/OIP-_no9YX5VYCocYJjQvShDbAHaGH.jpeg
./training-data/spider/OIP-f8rJQtxoP5RdG-dsrknuVgHaF9.jpeg
./training-data/spider/OIP-96TZ-FTXb8_40RuYTYhmkAHaFt.jpeg
./training-data/spider/OIP-Xrd-4i5V0C_tUUfyTzcLDgHaE8.jp

./training-data/spider/OIP-ygbe4i0-1ewbEAI8aiV6kQHaE4.jpeg
./training-data/spider/OIP-crNX8503-QXvpGeGJjpopQHaFj.jpeg
./training-data/spider/OIP-Vo7oj39_hrp5z8N7NWxwYgAAAA.jpeg
./training-data/spider/OIP-SPx0POO4f51gfLh44TcGJwHaE8.jpeg
./training-data/spider/OIP-YQXaeXxaHofNUT_VI4ZyXgHaDB.jpeg
./training-data/spider/OIP-Xp9H58fGNbmq3FJ0IeW3rgHaGi.jpeg
./training-data/spider/OIP-g7GYJ_iQv4e1hEVrWfwLKwHaH4.jpeg
./training-data/spider/OIP-J19GpabhHSQlWsVN0gWxGQHaE7.jpeg
./training-data/spider/OIP-gcqTrIrAUmy6eyP7mCUqNgHaFp.jpeg
./training-data/spider/OIP-ZUnieU4McT-8jIM1p_Wy_QHaIl.jpeg
./training-data/spider/OIP-kTX3pC6rm5BczJ4o-HDhrgHaHa.jpeg
./training-data/spider/OIP-gCZwK-nLRxB5U0Ylqp46tAHaEo.jpeg
./training-data/spider/OIP-ErO5a1ei5hghPK69Lp_6OAHaFj.jpeg
./training-data/spider/eb32b20e2cfd053ed1584d05fb1d4e9fe777ead218ac104497f5c97ca5ecb3b9_640.jpg
./training-data/spider/OIP-zkYZztHK-caFuMIxwMiqkAHaE6.jpeg
./training-data/spider/OIP-qBmvFSkUnFALLh30t1SUEAHaF9.jpeg
./training-data/spi

./training-data/spider/OIP-hSnFGvBxOAHdfL3V_pULPgHaFj.jpeg
./training-data/spider/OIP-tNaRnCzFQeFZGdRXwo50aAHaE8.jpeg
./training-data/spider/OIP-bzAXVtYCm1M275nd8jpF2AHaFj.jpeg
./training-data/spider/OIP-4v1DayFbeG7abHVhCL2W9QHaE7.jpeg
./training-data/spider/OIP-PqwbmQfvVwX7BGz4Iv3FBgAAAA.jpeg
./training-data/spider/OIP-IEGET7Pc5fkboBLTz6r7fwHaE8.jpeg
./training-data/spider/OIP-fIE16CvxaHeE7iw3hk7XzgHaF7.jpeg
./training-data/spider/OIP-1p4IuddkZuB5R0ZAfp2I_wHaKm.jpeg
./training-data/spider/OIP-0KrFxjWmN86M_XIKlWwg-QHaE8.jpeg
./training-data/spider/OIP-kH_MuXdOjUUG8x5hZCzUnQHaG1.jpeg
./training-data/spider/OIP-__HPppHyk7d0izZybMZ9bgAAAA.jpeg
./training-data/spider/OIP-RyBGZuOEeZNn6o8tbEwTVwHaFi.jpeg
./training-data/spider/OIP-pq-Dpyww0jITwDnHLD5w-QHaFx.jpeg
./training-data/spider/OIP-TcSE6XcbMCn07C50J_w6HQHaFj.jpeg
./training-data/spider/OIP-LK7d86__Q-rFlhlVdaPEHwHaHa.jpeg
./training-data/spider/OIP-vGzyOuclqPHrY9823vouqQHaE7.jpeg
./training-data/spider/OIP-4l6dVZt5TmH0ul5dFiQ6uQHaHc.jp

./training-data/cow/OIP-3Lg0oD4z3v_NrCLgfyOreAHaFi.jpeg
./training-data/cow/OIP--ojaB8xSPBPAxcGSMxrToQHaF7.jpeg
./training-data/cow/OIP-exyEuglzHYW3qcGVl8e4egHaEr.jpeg
./training-data/cow/OIP-i8b9pR47yJNxlFcmUq9SyQHaEc.jpeg
./training-data/cow/OIP-RTPZQue2HtimZfLLAQsZzAHaEE.jpeg
./training-data/cow/OIP-1hSIgzGdBweAlnjTuQVZ6AHaKG.jpeg
./training-data/cow/OIP-E4dXh-7HPSNK4IhJmk6l0AHaEy.jpeg
./training-data/cow/OIP-dx2dSg1gvTb08yl9cUDWqAHaE7.jpeg
./training-data/cow/OIP-2laahR5mKQFNkBu-JNRbxAHaHz.jpeg
./training-data/cow/OIP-8CEM_C2LhLfrdZLcoQ6TMQHaFj.jpeg
./training-data/cow/OIP-AzYAjAS9DR4Z6qqp2m5SUgHaFl.jpeg
./training-data/cow/OIP-j9X396YrR4Ogbm9zNwAknAHaKe.jpeg
./training-data/cow/OIP-BDav4pbJA56MVv0kSqPxPAHaFj.jpeg
./training-data/cow/OIP-wilcpy1nOuylDxcSmsux1wAAAA.jpeg
./training-data/cow/OIP-T0tSbCnGc0pGc1ja5c7XfgHaHa.jpeg
./training-data/cow/OIP-ETFRth73H_o90aZAy0JRrwHaEo.jpeg
./training-data/cow/OIP-RRn7M4sRzlRCJv3IH1V0oAHaFj.jpeg
./training-data/cow/OIP-YnOgPF6iyEbmrPr_mG8KhAHa

./training-data/cow/OIP-aPLSp_ySvpCv_JY33uffTgHaFc.jpeg
./training-data/cow/OIP-OkhbgyklzkTGwioFETJgkwHaE1.jpeg
./training-data/cow/OIP-uS0W5UR8obA-5xZRo1fXZQHaFj.jpeg
./training-data/cow/OIP-I6zZ6pMJIX3THKnlBUGCsgHaE8.jpeg
./training-data/cow/OIP-Jy1jbDqy8-AjaeEkOSSmjQHaFj.jpeg
./training-data/cow/OIP-1dcLlGo-oFc6UqTUl6yeHwHaEt.jpeg
./training-data/cow/OIP-fghrlZBPhAZZWn7H4RRXkgHaLm.jpeg
./training-data/cow/OIP-lYz1OKvwxL9fRh-iddSmGAHaDj.jpeg
./training-data/cow/OIP-8fITi6odS5d7nFFrt3ZVkgHaEK.jpeg
./training-data/cow/OIP-AqhBSJv6wtCP3vZrKFDsgQHaE7.jpeg
./training-data/cow/OIP-pBpo_9uRCpeTXcpDT2ZNwAHaLI.jpeg
./training-data/cow/OIP-t4WbVAv1BMiYlxS_vz_GxwHaFj.jpeg
./training-data/cow/OIP-hnBbkVBPcRZRkxX5btSiTwHaFj.jpeg
./training-data/cow/OIP-hnSLholxU3YAGnAAMeZRmwHaE8.jpeg
./training-data/cow/OIP-rLmXVnErpAn3aeQGjECe4QHaFk.jpeg
./training-data/cow/OIP-7vpStw3rU2tEQSYbTX9rNQAAAA.jpeg
./training-data/cow/OIP-YaG-7zWoe_kY8_E5JDS5BgHaEK.jpeg
./training-data/cow/OIP-vcXcbQMT6R3RxpGPE1nSlAAA

...

./training-data/squirrel/OIP-CBmRVQJX_6kUet8NsqHlVQHaFk.jpeg
./training-data/squirrel/OIP-bCn2U9HSSZbJVCNGmalBmAHaG2.jpeg
./training-data/squirrel/OIP-sEvyILYDpvW6pbUj8-h43wAAAA.jpeg
./training-data/squirrel/OIP-tRPpY8FFNBDM7Vu3EshfqQHaLJ.jpeg
./training-data/squirrel/OIP-V3tt7Vgd38E2D6mNINdEWAHaEo.jpeg
./training-data/squirrel/OIP-aon1m7Lw7gg1iQpf32cJFgHaLH.jpeg
./training-data/squirrel/OIP-LqBKns-JyfvnH-OjoIFUKwHaG6.jpeg
./training-data/squirrel/OIP-iLaOyV95KDJJm5QzIHt3RQHaLH.jpeg
./training-data/squirrel/OIP-nO9vaILQGjpyvzPQedimvgHaE5.jpeg
./training-data/squirrel/OIP-817778tPKZPZ3f1azpMDFwHaE6.jpeg
./training-data/squirrel/OIP-lutQilupGQvKQU6Id6qeSwHaFd.jpeg
./training-data/squirrel/OIP-H530I0pMlkihPzbDEZeLtwHaDr.jpeg
./training-data/squirrel/OIP-gmIenPqs_RB26bn_2WZsgwHaF7.jpeg
./training-data/squirrel/OIP-65P-8keEuRChU3XS_vi4xgHaFu.jpeg
./training-data/squirrel/OIP-TaJVqEiBbQLX000G8UKbdgHaH6.jpeg
./training-data/squirrel/OIP-UPBqbmKc9JYrDOccd07ZtwHaFc.jpeg
./training-data/squirrel

./training-data/squirrel/OIP-ih_MX32iP_9vXv6gTjLj-wHaFD.jpeg
./training-data/squirrel/OIP-rVQrfhkOHes45HBGQVUMTgHaE5.jpeg
./training-data/squirrel/OIP-U-BBnP2gmFXRTkq9UiwUKAHaIK.jpeg
./training-data/squirrel/OIP-_GXcWuoMOJ-aLnFOH9U52gAAAA.jpeg
./training-data/squirrel/OIP-FclAID2QFeaOgv8kbu4cPgHaEX.jpeg
./training-data/squirrel/OIP-Zgpal1X51YpES2SXbO2eQgHaFs.jpeg
./training-data/squirrel/OIP-M_IErIQdNaFyBFeUZ1HHRgHaFn.jpeg
./training-data/squirrel/OIP-vWovTQPSWojyZaQPTevVaAAAAA.jpeg
./training-data/squirrel/OIP-FO-Wun6SR51l84dekj6GuAHaFV.jpeg
./training-data/squirrel/OIP-D1ECpforuq6QEJCzzlUsUAHaE7.jpeg
./training-data/squirrel/OIP-_7pz-4WgSloa6KRjq6DozAHaFj.jpeg
./training-data/squirrel/OIP-JmhiJL90oZ7EgYmSd6vcnAHaE8.jpeg
./training-data/squirrel/OIP-hVEt1HjSZXSmkPMaEALyxAHaE8.jpeg
./training-data/squirrel/OIP-p_dE_q__i0YBGckfsOY2ZAHaF4.jpeg
./training-data/squirrel/OIP-2LuphON3Pki_UkxF105hVwHaE7.jpeg
./training-data/squirrel/OIP-ksIw2WAlsAhEQe9G2D6F5gAAAA.jpeg
./training-data/squirrel

./training-data/squirrel/OIP-QfuDZVlsH1gubUfnzrCVrAHaJ4.jpeg
./training-data/squirrel/OIP-gKlckLC_u_1gGkh7Zh5KkgHaE7.jpeg
./training-data/squirrel/OIP-QfErVP1nQGPruTtHXlcZ9AHaE8.jpeg
./training-data/squirrel/OIP-f0BoB-JU1GbYroE7IHC6ywHaC7.jpeg
./training-data/squirrel/OIP-hj6QX1eBD3gSzdlUTiiTaAHaE8.jpeg
./training-data/squirrel/OIP-lchQhEMT7yBud5vilf5VJgHaHz.jpeg
./training-data/squirrel/OIP-LL_ixhWJVqmMW1hHcQelXwAAAA.jpeg
./training-data/squirrel/OIP-lBSGPpzF5fFczWsxk3PGbQHaE8.jpeg
./training-data/squirrel/OIP-OOM7q9tGu_aiCNGwD13HHwHaFa.jpeg
./training-data/squirrel/OIP-Lk7WcTONlu_6iuoWkIY2CAHaEo.jpeg
./training-data/squirrel/OIP-Az-yaRC-q6mt6YqvP-yPogHaE7.jpeg
./training-data/squirrel/OIP-d6FQ9oA9k0E8bVo_vUCaSAHaHa.jpeg
./training-data/squirrel/OIP-AEVFXQ_yUFEAK14joNroCwHaIr.jpeg
./training-data/squirrel/OIP-KdjZQVRav5CijB8rN2quzAHaE8.jpeg
./training-data/squirrel/OIP-OOLX7PkPf2XapORkhBlkPAHaGn.jpeg
./training-data/squirrel/OIP-FpM9xWeKA8nHW2y8s4zd8AHaE7.jpeg
./training-data/squirrel

./training-data/horse/OIP-mIyiKZImUWDuxlwQayFXtAHaFj.jpeg
./training-data/horse/OIP-qvDtsbZZpDZFMcM2odmH6QHaGC.jpeg
./training-data/horse/OIP-YIoPgxBZWErbRZpMYBwiSgHaEK.jpeg
./training-data/horse/OIP-1PC1FRrJVWWQfhpV-hGaMAHaJ4.jpeg
./training-data/horse/OIP-V6cAvwr6vkbFqn0BW9qj3QHaE7.jpeg
./training-data/horse/OIP-a4B95uBVG_lr1sQZpcjBtgHaFj.jpeg
./training-data/horse/OIP-JFZaad4LzvXLIcom6-PipAHaJ4.jpeg
./training-data/horse/OIP-8aRoCbatU4dierBCZ5VjYAHaFj.jpeg
./training-data/horse/OIP-tzYoBhV-tvJBG9wWZbUkxQHaK0.jpeg
./training-data/horse/OIP-Aoz-SavxW0pt9ljXedqPrQHaFu.jpeg
./training-data/horse/OIP-imEgzeGprjGVfXtQP_8EEAHaEs.jpeg
./training-data/horse/OIP-wEZ5Hm77hBdVd-qIFf7h8gHaFj.jpeg
./training-data/horse/OIP-UElTwXZBi4obQhYzPK_u6AHaJ4.jpeg
./training-data/horse/OIP-d9-20UlLLtQDAMODxTY6TQHaGW.jpeg
./training-data/horse/OIP-29sN2KSWNlkpMSXp_m6GVAAAAA.jpeg
./training-data/horse/OIP-F0rwRujAhQ69ciOtv1nkVwHaFL.jpeg
./training-data/horse/OIP-KOL6lSWC_eHCNnMmym97LgHaE9.jpeg
./training-dat

./training-data/horse/OIP-b0Vh47DW5lkt7HK5wpVtfwHaFn.jpeg
./training-data/horse/OIP-IjQKY3BhfycCgvEW6m-6mgHaJj.jpeg
./training-data/horse/OIP-_qNBuQS2EXVb5478ITl5wAAAAA.jpeg
./training-data/horse/OIP-GmMcLQXDX352XrZ1TXxz9QHaE8.jpeg
./training-data/horse/OIP-dVOlx9d96XGjDyse5Nt5PQHaE8.jpeg
./training-data/horse/OIP-KPzL7Wjn07iKHJTKOwk6pAHaE_.jpeg
./training-data/horse/OIP-OIciJ10rB-I1fiUt7UL9MAD6D6.jpeg
./training-data/horse/OIP-VVh_bA9absUFvQgkhspXFwHaFj.jpeg
./training-data/horse/OIP-U0SRxeUYnBGSJhb_YzLGCQHaFj.jpeg
./training-data/horse/OIP-07fmgenGGDx8ppNm-y4e-AHaE7.jpeg
./training-data/horse/OIP-H6FoNEdOIESxQ48fYsdpvQHaE0.jpeg
./training-data/horse/OIP-HVmTIAQKHBA0s_Zz-GDR9AHaJQ.jpeg
./training-data/horse/OIP-K71X2XaN36FJAJVRs9x8nwHaFy.jpeg
./training-data/horse/OIP-VH8_cl48bmfWXQijJoUAzQHaEJ.jpeg
./training-data/horse/OIP-eLv4fZhi9uN4MfW5RvOy8QHaE8.jpeg
./training-data/horse/OIP-3YlxWp97bTbNFBAVTJuLZQHaKT.jpeg
./training-data/horse/OIP-yZ5vvcPPiK5plZD_xa8dvQHaEK.jpeg
./training-dat

In [5]:
j = 100
test_images = []

a = images[j]
img = np.array(a)
img = img[:, :, ::-1].copy() 
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
if(img.shape[0] > img.shape[1]):
    tile_size = (int(img.shape[1]*256/img.shape[0]),256)
else:
    tile_size = (256, int(img.shape[0]*256/img.shape[1]))
img = centering_image(cv2.resize(img, dsize=tile_size))
img = img[16:240, 16:240]
test_images.append(img)

test_images = np.array(test_images, dtype='f').reshape(-1,224,224,3)

data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:1].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/saved_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print(predictions)


Data: {"signature_name": "serving_default", "instances": ... 0], [214.0, 184.0, 157.0], [220.0, 190.0, 162.0]]]]}
[[6.6497613e-11, 2.25502236e-15, 3.53290701e-19, 8.51250046e-17, 1.91997019e-22, 2.09682833e-21, 1.0, 1.43942204e-13, 6.18397274e-14, 6.14086923e-26]]
